In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
np.random.seed(0)

## Preparation

Let's first create a simple distribution $p(y|x)$. We shall use a simple model
$$
p(y|x) = \mathcal{N}(y|\sin(\pi x), 1).
$$
In other words, the distribution is a Gaussian centered around $\mu = \sin (\pi x)$ with unit variance.